<a href="https://colab.research.google.com/github/chanakya-bansal/pokemon_color_mapping/blob/main/pokemon_color_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil
import requests
import csv
from bs4 import BeautifulSoup
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import datetime

# Download latest version
path = kagglehub.dataset_download("kvpratama/pokemon-images-dataset")

print("Path to dataset files:", path)

100%|██████████| 70.9M/70.9M [00:04<00:00, 18.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kvpratama/pokemon-images-dataset/versions/2


In [ ]:

# Define the destination directory
destination = "/content/pokemon_dataset"

# Move the dataset
shutil.move(path, destination)

print("Dataset moved to:", destination)


Dataset moved to: /content/pokemon_dataset


In [ ]:
def get_pokemon_color(pokemon_name):
    url = f"https://pokeapi.co/api/v2/pokemon-species/{pokemon_name.lower()}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data["color"]["name"]
    else:
        return None


In [ ]:

# URL of the website to scrape
URL = "https://pokemondb.net/pokedex/national"

# Send a GET request
response = requests.get(URL)
soup = BeautifulSoup(response.text, "html.parser")

# Find all Pokémon data containers
pokemon_data = []
pokemon_cards = soup.find_all("div", class_="infocard")

for card in pokemon_cards:
    number = card.find("small").text.strip("#")  # Extract Pokémon number
    name = card.find("a", class_="ent-name").text  # Extract Pokémon name
    color=get_pokemon_color(name)
    pokemon_data.append([number, name,color])

# Save data to CSV
csv_filename = "pokemon_list.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Number", "Name","color"])  # CSV header
    writer.writerows(pokemon_data)

print(f"Scraped {len(pokemon_data)} Pokémon and saved to {csv_filename}")


Scraped 1025 Pokémon and saved to pokemon_list.csv


In [ ]:


def view_random_image_with_name(image_folder, csv_file):
    # Load CSV file
    df = pd.read_csv(csv_file)

    # Standardize column names
    df.columns = df.columns.str.strip().str.lower()

    # Ensure 'number' column exists
    if 'number' not in df.columns:
        print(" 'number' column not found in CSV. Check column names!")
        return None

    # Remove leading zeros from CSV numbers
    df['number'] = df['number'].astype(str).str.lstrip("0")

    # Get all image filenames
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]

    if not image_files:
        print("No images found in the folder.")
        return None

    # Choose a random image
    random_image = random.choice(image_files)
    pokemon_number = os.path.splitext(random_image)[0]  # Extract number

    # Remove leading zeros from the image number as well
    pokemon_number = pokemon_number.lstrip("0")

    # Debugging prints
    #print("First few numbers in CSV:", df["number"].head(10).tolist())
    print(f"Extracted Image Number: {pokemon_number}")

    # Find Pokémon name in CSV
    pokemon_info = df[df['number'] == pokemon_number]

    if pokemon_info.empty:
        print(f" No matching Pokémon found for number {pokemon_number}")
        return None

    pokemon_name = pokemon_info.iloc[0]['name']
    pokemon_color=get_pokemon_color(pokemon_name)
    # Load and display the image
    image_path = os.path.join(image_folder, random_image)
    img = mpimg.imread(image_path)
    plt.imshow(img)
    plt.title(f"Pokémon: {pokemon_name} (#{pokemon_number})")
    plt.axis("off")
    plt.show()

    print(f"Image Filename: {random_image}")
    print(f"Pokémon Name: {pokemon_name}")
    print(f"Pokemon color: {pokemon_color}")
    print(f"Image Shape: {img.shape}")

    #return img


In [ ]:
view_random_image_with_name("/content/pokemon_dataset/pokemon/pokemon","/content/pokemon_list.csv")


Extracted Image Number: 130-mega
 No matching Pokémon found for number 130-mega


In [ ]:
# Load CSV
df = pd.read_csv("pokemon_list.csv")

# Convert 'color' column to a list
class_names = df["color"].unique().tolist()
class_names.pop(7)
print(class_names)

['green', 'red', 'blue', 'white', 'brown', 'yellow', 'purple', 'pink', 'gray', 'black']


In [ ]:
image_folder = "/content/pokemon_dataset/pokemon/pokemon"
output_folder = "train_data"
csv_path = "pokemon_list.csv"

# Load CSV
df = pd.read_csv(csv_path)

# Loop through dataset
for index, row in df.iterrows():
    # Get image name (Ensure number format matches file names)
    image_name = str(int(row["Number"])) + ".png"  # Convert properly to int first
    color = str(row["color"])  # Use color as label

    # Define folder for this label
    label_folder = os.path.join(output_folder, color)
    os.makedirs(label_folder, exist_ok=True)

    # Define source and destination paths
    src_path = os.path.join(image_folder, image_name)
    dest_path = os.path.join(label_folder, image_name)

    # Move or copy the image if it exists
    if os.path.exists(src_path):
        shutil.copy(src_path, dest_path)  # Use shutil.copy() if you don't want to remove the original
    else:
        print(f"Image not found: {src_path}")

print("Sorting complete!")

Image not found: /content/pokemon_dataset/pokemon/pokemon/1.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/2.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/3.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/4.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/5.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/6.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/7.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/8.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/9.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/10.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/11.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/12.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/13.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/14.png
Image not found: /content/pokemon_dataset/pokemon/pokemon/15.png
Image not found: /content/pokemon_

In [ ]:
# splitting the data into training and validation sets and preprocessing into tensorflow datasets
IMG_SIZE=(256,256)
train_dir="train_data"

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_dir,
    image_size=IMG_SIZE,
    label_mode="categorical",
    batch_size=32,
    validation_split=0.2,
    subset="training",
    seed=42
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    seed=42,
    directory=train_dir,
    image_size=IMG_SIZE,
    label_mode="categorical",
    batch_size=32,
    validation_split=0.2,
    subset="validation"

)




Found 697 files belonging to 11 classes.
Using 558 files for training.
Found 697 files belonging to 11 classes.
Using 139 files for validation.


In [ ]:
# a tensorboard callback
def create_tensorboard(dir_name,exp_name):

  log_dir=dir_name+'/'+exp_name+'/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

  tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir)

  print(f"saving tensorboard log files to {log_dir}")

  return tensorboard_callback


In [ ]:
model_1=tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(filters=20,kernel_size=4,activation="relu",
                           input_shape=(256,256,3)),
    tf.keras.layers.Conv2D(20,4,activation="relu"),

    tf.keras.layers.MaxPool2D(pool_size=2,padding="valid"),

    tf.keras.layers.Conv2D(20,4,activation="relu"),
    tf.keras.layers.Conv2D(20,4,activation="relu"),

    tf.keras.layers.MaxPool2D(pool_size=2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(11,activation="softmax")


])

model_1.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])




hist_1=model_1.fit(train_data,
          epochs=5,
          steps_per_epoch=len(train_data),
          validation_data=val_data,
          validation_steps=int(len(val_data)*0.25),
          callbacks=[create_tensorboard("tensorflow_hub",
                                               "model_1")])

saving tensorboard log files to tensorflow_hub/model_1/20250330-131632
Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 18s 467ms/step - accuracy: 0.1307 - loss: 47.3922 - val_accuracy: 0.1250 - val_loss: 2.7918
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 0.2986 - loss: 2.2644 - val_accuracy: 0.1875 - val_loss: 2.2221
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.3643 - loss: 2.0485 - val_accuracy: 0.1875 - val_loss: 2.2527
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.6999 - loss: 1.3338 - val_accuracy: 0.1562 - val_loss: 2.8879
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.9188 - loss: 0.4042 - val_accuracy: 0.2188 - val_loss: 3.6192


In [ ]:
from tensorflow.keras import layers, models

# Define CNN model
model = models.Sequential([
    # Convolutional layers
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(256, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    # Flatten layer
    layers.Flatten(),

    # Fully connected layers
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),

    # Output layer (assuming 10 colors, use softmax for multi-class classification)
    layers.Dense(11, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary
#model.summary()



hist_2=model.fit(train_data,
          epochs=20,
          steps_per_epoch=len(train_data),
          validation_data=val_data,
          validation_steps=int(len(val_data)*0.25))

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 222ms/step - accuracy: 0.0941 - loss: 106.2821 - val_accuracy: 0.1562 - val_loss: 2.3847
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 117ms/step - accuracy: 0.1262 - loss: 2.3492 - val_accuracy: 0.1875 - val_loss: 2.3317
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step - accuracy: 0.2323 - loss: 2.1794 - val_accuracy: 0.1562 - val_loss: 2.3337
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.3427 - loss: 1.9067 - val_accuracy: 0.2500 - val_loss: 2.3402
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 136ms/step - accuracy: 0.3842 - loss: 1.6747 - val_accuracy: 0.2500 - val_loss: 2.4838
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 136ms/step - accuracy: 0.5573 - loss: 1.3525 - val_accuracy: 0.1875 - val_loss: 2.3997
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - accuracy: 0.5998 - loss: 1.2392 - val_accuracy: 0.1562 - val_loss: 2.7048
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 139ms/step - accuracy: 0.6618 - loss: 1.0420 - val_accuracy: 0.

In [ ]:


data_aug=tf.keras.Sequential([
    tf.keras.layers.Rescaling(1/255.),
   # tf.keras.layers.RandomFlip("horizontal"),
    #tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomContrast(0.05),
],name="data_aug")


conv_block_1=tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=30, kernel_size=2, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(30, 3, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=2, padding="valid"),
],name="conv_block_1")

conv_block_2=tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=30, kernel_size=2, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(30, 3, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=2, padding="valid"),
],name="conv_block_2")

flatten_block=tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.BatchNormalization(),

],name="flatten_block")


inputs=tf.keras.layers.Input(shape=(256,256,3),name="input_layer")
x=data_aug(inputs)
x=conv_block_1(x)
x=conv_block_2(x)
x=flatten_block(x)
outputs=tf.keras.layers.Dense(11,activation="softmax",name="output_layer")(x)

model_2=tf.keras.Model(inputs,outputs)


model_2.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


hist_2 = model_2.fit(
    train_data,
    epochs=5,
    steps_per_epoch=len(train_data),
    validation_data=val_data,
    validation_steps=int(len(val_data) * 0.25),
   # callbacks=[create_tensorboard("tensorflow_hub","model_2")]
)


Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 473ms/step - accuracy: 0.2853 - loss: 2.5498 - val_accuracy: 0.0938 - val_loss: 3.0269
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 224ms/step - accuracy: 0.8664 - loss: 0.5546 - val_accuracy: 0.0312 - val_loss: 4.5339
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 117ms/step - accuracy: 0.9615 - loss: 0.2496 - val_accuracy: 0.1875 - val_loss: 4.2020
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step - accuracy: 1.0000 - loss: 0.0987 - val_accuracy: 0.0938 - val_loss: 7.5839
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - accuracy: 1.0000 - loss: 0.0462 - val_accuracy: 0.1875 - val_loss: 7.3694


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50

# Define the input shape and number of classes
input_shape = (256, 256, 3)
num_classes = 11

# Input layer
inputs = tf.keras.Input(shape=input_shape)

# Preprocess inputs: Rescale pixel values to [0,1]
x = layers.Rescaling(1./255)(inputs)

# Load pre-trained ResNet50 without the top layers
# Set weights='imagenet' for transfer learning and include_top=False to remove classification head
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = False  # Freeze the base model to retain pre-trained features

# Pass the inputs through the base model
x = base_model(x, training=False)

# Global average pooling to reduce spatial dimensions
x = layers.GlobalAveragePooling2D()(x)

# Add a dropout layer for regularization
x = layers.Dropout(0.5)(x)

# Fully connected (Dense) layer to generate predictions for 11 classes
outputs = layers.Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Now you can compile the model with your chosen optimizer, loss, and metrics:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


hist = model.fit(
    train_data,
    epochs=5,
    steps_per_epoch=len(train_data),
    validation_data=val_data,
    validation_steps=int(len(val_data) * 0.25),
   # callbacks=[create_tensorboard("tensorflow_hub","model_2")]
)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 26s 624ms/step - accuracy: 0.1283 - loss: 2.7557 - val_accuracy: 0.1562 - val_loss: 2.3166
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 110ms/step - accuracy: 0.1268 - loss: 2.5703 - val_accuracy: 0.1875 - val_loss: 2.1262
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.1290 - loss: 2.4557 - val_accuracy: 0.2500 - val_loss: 2.0777
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 138ms/step - accuracy: 0.1829 - loss: 2.3613 - val_accuracy: 0.1875 - val_loss: 2.2168
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 109ms/step - accuracy: 0.1690 - loss: 2.2780 - val_accuracy: 0.2188 - val_loss: 2.1786


In [ ]:
import cv2
import numpy as np
import pandas as pd
from glob import glob
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# ----- 1. Load the CSV with image filenames and dominant color labels -----
# The CSV is expected to have at least two columns: "Number" and "color"
df = pd.read_csv("pokemon_list.csv")  # Adjust the CSV filename/path accordingly

features = []
labels = []

# ----- 2. Feature Extraction: Compute a Hue and Saturation Histogram for Each Image -----
for index, row in df.iterrows():
    # Skip images based on criteria
    if row["Number"] > 721 or row["color"] == "black" or str(row["color"]).lower() == "nan":
        continue

    image_path = "/content/pokemon_dataset/2/pokemon/pokemon/" + str(row["Number"]) + ".png"  # Adjust path as needed
    label = row["color"]  # dominant color label, e.g., "yellow"

    # Load the image using OpenCV
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Image {image_path} not found or cannot be read.")
        continue

    # Convert the image from BGR (OpenCV default) to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Compute histogram for the Hue channel (channel 0) using 16 bins
    h_hist = cv2.calcHist([hsv], [0], None, [16], [0, 180])
    # Compute histogram for the Saturation channel (channel 1) using 16 bins
    s_hist = cv2.calcHist([hsv], [1], None, [16], [0, 256])

    # Concatenate the two histograms into one feature vector (32 features)
    hist = np.concatenate((h_hist, s_hist))
    # Normalize the histogram so that it sums to 1
    hist = cv2.normalize(hist, hist).flatten()

    features.append(hist)
    labels.append(label)

# Convert lists to NumPy arrays
X = np.array(features)
y = np.array(labels)

# ----- 3. Encode Labels and Split Data -----
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# ----- 4. Train a Classifier (Support Vector Machine) -----
# You can experiment with kernel='rbf' if needed.
clf = SVC(kernel='linear', probability=True, random_state=42)
clf.fit(X_train, y_train)

# ----- 5. Evaluate the Model -----
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Test Accuracy: 0.4233576642335766
Classification Report:
              precision    recall  f1-score   support

        blue       0.34      0.95      0.50        22
       brown       0.41      0.88      0.56        25
        gray       0.00      0.00      0.00        21
       green       0.90      0.64      0.75        14
        pink       1.00      0.22      0.36         9
      purple       0.50      0.33      0.40         9
         red       0.00      0.00      0.00        16
       white       0.00      0.00      0.00         9
      yellow       0.33      0.08      0.13        12

    accuracy                           0.42       137
   macro avg       0.39      0.35      0.30       137
weighted avg       0.35      0.42      0.32       137



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
